In [1]:
__author__ = "konwar.m"
__copyright__ = "Copyright 2022, AI R&D"
__credits__ = ["konwar.m"]
__license__ = "Individual Ownership"
__version__ = "1.0.1"
__maintainer__ = "konwar.m"
__email__ = "rickykonwar@gmail.com"
__status__ = "Development"

### Importing Libraries

In [2]:
import os
import pickle
import pandas as pd
from tqdm import tqdm

In [3]:
os.chdir('..')
os.getcwd()

'c:\\Users\\manash.jyoti.konwar\\Documents\\AI_Random_Projects\\ML-Retail-Sales'

### Loading Training Data and Features

In [4]:
finalized_train_dict, finalized_test_dict = {}, {}

In [5]:
train_test_path = r'datasets\train_test_datasets_v02'
files = os.listdir(train_test_path)

training_files = [i for i in files if i.endswith('.csv') and i.startswith('train')]
testing_files = [i for i in files if i.endswith('.csv') and i.startswith('test')]

with open(os.path.join('datasets', 'train_test_datasets_v02', 'parent_cat_map.pkl'), "rb") as fp:
    parent_category_map = pickle.load(fp)

parent_category_inv_map = {v: k for k, v in parent_category_map.items()}

In [6]:
parent_category_inv_map

{0: 'Accessories',
 1: 'Android Games',
 2: 'Books',
 3: 'Cinema',
 4: 'Clean Media (Piece)',
 5: 'Clean Media (Spire)',
 6: 'Delivery Of Goods',
 7: 'Film',
 8: 'Games',
 9: 'Gaming Consoles',
 10: 'Gifts',
 11: 'Mac Games',
 12: 'Music',
 13: 'Pc',
 14: 'Pc Games',
 15: 'Payment Cards (Cinema, Music, Games)',
 16: 'Payment Cards',
 17: 'Power Elements',
 18: 'Programs',
 19: 'Service',
 20: 'Tickets (Digit)',
 21: 'Игры'}

In [7]:
for train_file in tqdm(training_files, desc='Loading training files'):
    finalized_train_dict[parent_category_inv_map.get(int(train_file.split('_')[1]))] = pd.read_csv(os.path.join('datasets','train_test_datasets_v02',train_file))

for test_file in tqdm(testing_files, desc='Loading testing files'):
    finalized_test_dict[parent_category_inv_map.get(int(train_file.split('_')[1]))] = pd.read_csv(os.path.join('datasets','train_test_datasets_v02',test_file))    

Loading testing files: 100%|██████████| 21/21 [00:01<00:00, 16.03it/s]
